In [19]:
import findspark
findspark.init()
import pandas as pd
from pyspark import SparkConf, SparkContext
import os
import re
import time
import datetime
import json

conf = SparkConf()
sc = SparkContext.getOrCreate(conf=conf)

In [20]:
K = 160
       
with open('./data_{}_cluster/user_feature_{}.json'.format(K, K), 'r') as inputfile:  
    user_feature_dict = json.load(inputfile)

user_ids = user_feature_dict.keys()
print(len(user_feature_dict))

7040


In [21]:
print(user_feature_dict.get('122b7665-6fac-4278-97e8-b72f7ccbb5dc'))

[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.023809523809523808, 0.0, 0.0, 0.0, 0.0, 0.047619047619047616, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.047619047619047616, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.023809523809523808, 0.0, 0.023809523809523808, 0.0, 0.0, 0.0, 0.0, 0.047619047619047616, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.023809523809523808, 0.023809523809523808, 0.0, 0.0, 0.11904761904761904, 0.0, 0.0, 0.09523809523809523, 0.07142857142857142, 0.0, 0.0, 0.0, 0.0, 0.023809523809523808, 0.0, 0.0, 0.023809523809523808, 0.023809523809523808, 0.0, 0.0, 0.0, 0.19047619047619047, 0.0, 0.0, 0.19047619047619047, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.

In [22]:
with open('./data_{}_cluster/post_feature_{}.json'.format(K, K), 'r') as inputfile:  
    post_feature_dict = json.load(inputfile)

post_ids = post_feature_dict.keys()
print(len(post_feature_dict))

61507


In [6]:
# generate post lists
post_list = post_feature_dict.keys()

print(len(post_list))

61507


In [6]:
user_click = sc.parallelize('')
for i in range(1, 10, 1):
    file_path = './userID_clickID_date_all/user_click_20180' + str(i)
    user_click_onemonth = sc.textFile(file_path).map(lambda line: tuple(re.findall(r"'(.*?)'", line)))\
                    .map(lambda line: (line[0].split('|')[0], line[1], line[2]))\
                    .filter(lambda x: user_feature_dict.get(x[0]) != None)\
    
    user_click = user_click.union(user_click_onemonth)

In [7]:
# add user feature
user_feature_click = user_click.map(lambda x: (x[0], x[1], x[2], user_feature_dict.get(x[0])))

#print(user_feature_click.take(2))

In [8]:
# add post feature
user_usrfeature_pstfeature = user_feature_click.filter(lambda x: post_feature_dict.get(x[1]) != None)\
                            .map(lambda x: (x[0], x[1], x[2], x[3], post_feature_dict.get(x[1])))

#print(user_usrfeature_pstfeature.take(1))

In [9]:
# calculate time diff
def convert_to_stp(date):
    date = date.split('-')
    date = date[1] + '/' + date[2] + '/20' + date[0][2:]
    return time.mktime(datetime.datetime.strptime(date, "%m/%d/%Y").timetuple())

user_usrfeature_pstfeature_output = user_usrfeature_pstfeature\
                    .map(lambda x: (x[0], x[1], convert_to_stp(x[2].split(' ')[0]), x[3], x[4]))\
                    .map(lambda x: (x[0], x[1], x[3], x[4][:-2], x[2] - float(x[4][-1])))\
                    .filter(lambda x: x[4] > 0)


#print(user_usrfeature_pstfeature_output.take(1))

In [10]:
user_usrfeature_pstfeature_train = user_usrfeature_pstfeature_output\
                .map(lambda x: (x[0], )+(x[1], )+ (x[4], ) +  tuple(x[2]) + tuple(x[3]))

#print(user_usrfeature_pstfeature_train.take(2))

In [11]:
from pyspark.sql.types import Row
from pyspark.sql import SparkSession
from pyspark import SparkContext


spark = SparkSession(sc)

#here you are going to create a function
def f(x):
    d = tuple()
    for i in range(len(x)):
        if i == 0 or i == 1:
            d += (x[i],)
        else:
            d += (float(x[i]),)
    return d

#Now populate that
df = user_usrfeature_pstfeature_train.map(lambda x: f(x)).toDF()



In [12]:
#df.show(1);

In [13]:
df.write.csv('./data_{}_cluster/training_1'.format(K))